In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# Advance Machine Learning 159.738  Assignment 1

---

### Team members:
Ziyi Wang ID 18042783

## Content
* Data cleaning inspection
* EDA
* 

---


#### Information about datasets
#### googleplaystore.csv
* App: Application name
* Category: Category the app belongs to
* Rating: Overall user rating of the app (as when scraped)
* Reviews: Number of user reviews for the app (as when scraped)
* Size: Size of the app (as when scraped)
* Installs: Number of user downloads/installs for the app (as when scraped)
* Type: Paid or Free
* Price: Price of the app (as when scraped)
* Content Rating: Age group the app is targeted at - Children / Mature 21+ / Adult
* Genres: An app can belong to multiple genres (apart from its main category). For eg, a musical family game will belong to Music, Game, Family genres.
* Last Updated: Date when the app was last updated on Play Store (as when scraped)
* Current Ver: Current version of the app available on Play Store (as when scraped)
* Android Ver: Min required Android version (as when scraped)

#### googleplaystore_user_reviews.csv
* App: Name of app
* Translated_Review: User review (Preprocessed and translated to English)
* Sentiment: Positive/Negative/Neutral (Preprocessed)
* Sentiment_Polarity: Sentiment polarity score
* Sentiment_Subjectivity: Sentiment subjectivity score

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import numpy as np
import patsy


%matplotlib inline

In [7]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

E:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.11.0


In [9]:
from pylab import rcParams

sns.set(style="ticks")
#sns.set_style("whitegrid")
rcParams['figure.dpi'] = 350
rcParams['lines.linewidth'] = 2
rcParams['axes.facecolor'] = 'white'
rcParams['patch.edgecolor'] = 'white'
rcParams['font.family'] = 'StixGeneral'
rcParams['figure.figsize'] = 10,8
rcParams['font.size'] = 20
rcParams['axes.labelsize'] = 'large'
rcParams['xtick.labelsize'] = 20
rcParams['ytick.labelsize'] = 20

pd.options.display.max_columns = 50

In [23]:
pd.options.display.max_columns = 150

#Read datasets
googleApp = pd.read_csv("googleplaystore.csv");
ga = pd.DataFrame(googleApp)
googleAppView = pd.read_csv("googleplaystore_user_reviews.csv")googleplaystore_user_reviews.csv
gav = pd.DataFrame(googleAppView)
googleApp.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [18]:
googleApp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
App               10841 non-null object
Category          10841 non-null object
Rating            9367 non-null float64
Reviews           10841 non-null object
Size              10841 non-null object
Installs          10841 non-null object
Type              10840 non-null object
Price             10841 non-null object
Content Rating    10840 non-null object
Genres            10841 non-null object
Last Updated      10841 non-null object
Current Ver       10833 non-null object
Android Ver       10838 non-null object
dtypes: float64(1), object(12)
memory usage: 1.1+ MB


In [16]:
googleAppView.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


In [19]:
googleAppView.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64295 entries, 0 to 64294
Data columns (total 5 columns):
App                       64295 non-null object
Translated_Review         37427 non-null object
Sentiment                 37432 non-null object
Sentiment_Polarity        37432 non-null float64
Sentiment_Subjectivity    37432 non-null float64
dtypes: float64(2), object(3)
memory usage: 2.5+ MB


In [21]:
googleApp.shape

(10841, 13)

In [22]:
googleAppView.shape

(64295, 5)

### 1.1 Data clean 
There are five cols data have NAN data each one is Rating, Type, Content Rating, Current Ver, Android Ver,Ratingdata is overall user rating of the app we useing mean value to replace the NAN data and Current Ver is current version of the app available on Play Store we replace it with zero. Type means paid or free we replace it to free and Content Rating is age group the app is targeted at - Children / Mature 21+ / Adult looking at detail information this data find it was a numerical fill error so we remove the data to correct place and this line's missing value would be Category and looking at name of App we replace value to Tools category and finally the Android Ver which is Min required Android version we replace it to latest vision of android. And for googleAppView dataframe we replace the NAN reviews to none.

In [36]:
# Number of missing in each column
missing = pd.DataFrame(ga.isnull().sum()).rename(columns = {0: 'total'})
missing

,total
App,0
Category,0
Rating,1474
Reviews,0
Size,0
Installs,0
Type,1
Price,0
Content Rating,1
Genres,0


In [46]:
for columname in ga.columns:
    if ga[columname].count() != len(ga):
        loc = ga[columname][ga[columname].isnull().values==True].index.tolist()
        print('Columns name："{}", the{}columns have missing value'.format(columname,loc))

Columns name："Rating", the[23, 113, 123, 126, 129, 130, 134, 163, 180, 185, 227, 321, 478, 479, 480, 610, 613, 617, 620, 621, 623, 624, 626, 627, 628, 629, 630, 631, 632, 635, 637, 638, 639, 641, 642, 643, 644, 647, 648, 649, 650, 651, 652, 653, 658, 659, 660, 666, 669, 683, 698, 704, 1013, 1024, 1025, 1032, 1033, 1034, 1039, 1041, 1043, 1045, 1046, 1177, 1180, 1455, 1469, 1478, 1484, 1489, 1517, 1519, 1520, 1530, 1534, 1536, 1537, 1539, 1540, 1552, 1557, 1559, 2111, 2265, 2280, 2294, 2356, 2390, 2419, 2422, 2423, 2424, 2425, 2430, 2431, 2432, 2433, 2434, 2435, 2436, 2437, 2438, 2439, 2440, 2441, 2442, 2443, 2446, 2447, 2449, 2451, 2453, 2456, 2460, 2461, 2462, 2465, 2466, 2467, 2468, 2469, 2470, 2471, 2473, 2474, 2478, 2479, 2480, 2481, 2483, 2485, 2498, 2501, 2502, 2503, 2513, 2516, 2529, 2540, 3252, 3258, 3577, 3579, 3580, 3583, 3585, 3589, 3599, 3603, 3616, 3621, 3705, 3959, 3985, 4130, 4134, 4137, 4139, 4177, 4201, 4218, 4256, 4265, 4269, 4270, 4273, 4278, 4303, 4307, 4308, 4315, 

In [54]:
#detail information about Content Rating 
ga.iloc[10472] 

App               Life Made WI-Fi Touchscreen Photo Frame
Category                                              1.9
Rating                                                 19
Reviews                                              3.0M
Size                                               1,000+
Installs                                             Free
Type                                                    0
Price                                            Everyone
Content Rating                                        NaN
Genres                                  February 11, 2018
Last Updated                                       1.0.19
Current Ver                                    4.0 and up
Android Ver                                           NaN
Name: 10472, dtype: object

In [55]:
#replace the missing value of Content Rating

In [32]:
# Create a percentage missing
missing['Rating'] = missing['total rating'] / len(ga)
missing.sort_values('Rating', ascending = False).head(10)

,total rating,Rating
Rating,1474,0.135965
Current Ver,8,0.000738
Android Ver,3,0.000277
Type,1,0.000092
Content Rating,1,0.000092
App,0,0.000000
Category,0,0.000000
Reviews,0,0.000000
Size,0,0.000000
Installs,0,0.000000
